<a href="https://colab.research.google.com/github/minjaekim070/Bootcamp_Project/blob/main/NLP_project_2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

프로젝트 목표 - 자연어 이해(NLU) 및 대화 모델을 활용하여 고객 지원 트위터 데이터 세트를 분석하고, 이를 바탕으로 고객 지원에 대한 통찰력을 얻고 대응 전략을 개발
Reference - https://www.kaggle.com/code/alexg99/text-processing-100530129

#데이터 업로드

In [1]:
!unzip drive/MyDrive/archive_twitter.zip

Archive:  drive/MyDrive/archive_twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


#Library

In [27]:
!pip install openai

In [28]:
import openai
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

#데이터 전처리

In [29]:
df = pd.read_csv('/content/twcs/twcs.csv')



In [30]:
sample_df = pd.read_csv('/content/sample.csv')

In [31]:
print(sample_df.shape, df.shape)

(93, 7) (2811774, 7)


In [32]:
print(sample_df.columns)
print(df.columns)

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')
Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')


트윗 식별자, 작성자의 식별자, 인바운드인지, 작성시간, 내용, 응답트윗 식별자, 어떤트윗의 응답인지를 나타내는 식별자
(결측치 있을 수 있음)

In [33]:
print(sample_df.iloc[0])

tweet_id                                                              119237
author_id                                                             105834
inbound                                                                 True
created_at                                    Wed Oct 11 06:55:44 +0000 2017
text                       @AppleSupport causing the reply to be disregar...
response_tweet_id                                                     119236
in_response_to_tweet_id                                                  NaN
Name: 0, dtype: object


In [34]:
has_null_values = sample_df.isnull

if has_null_values:
  print('결측치 있음')

결측치 있음


In [35]:
df_text=df[['text']]
df_text.head()

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


In [36]:
# text열을 모두 소문자로 변환
df_text['lower_text']=df_text['text'].str.lower()

# 결과 확인
df_text.head()

<ipython-input-36-85bf74d03cb8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['lower_text']=df_text['text'].str.lower()


,text,lower_text
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.


In [37]:
# Punctuation 제거
punc_to_remove=string.punctuation

# text 열에서 Punctuation 제거
def remove_punctuation(text):
    return text.translate(str.maketrans('','', punc_to_remove))

df_text["text_to_punc"]=df_text['text'].apply(lambda text: remove_punctuation(text))

<ipython-input-37-933f1af718ab>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text["text_to_punc"]=df_text['text'].apply(lambda text: remove_punctuation(text))


In [38]:
# stopwords(불용어) 다운로드
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
#영어 stopwords 제거
from nltk.corpus import stopwords
",".join(stopwords.words("english"))

"i,me,my,myself,we,our,ours,ourselves,you,you're,you've,you'll,you'd,your,yours,yourself,yourselves,he,him,his,himself,she,she's,her,hers,herself,it,it's,its,itself,they,them,their,theirs,themselves,what,which,who,whom,this,that,that'll,these,those,am,is,are,was,were,be,been,being,have,has,had,having,do,does,did,doing,a,an,the,and,but,if,or,because,as,until,while,of,at,by,for,with,about,against,between,into,through,during,before,after,above,below,to,from,up,down,in,out,on,off,over,under,again,further,then,once,here,there,when,where,why,how,all,any,both,each,few,more,most,other,some,such,no,nor,not,only,own,same,so,than,too,very,s,t,can,will,just,don,don't,should,should've,now,d,ll,m,o,re,ve,y,ain,aren,aren't,couldn,couldn't,didn,didn't,doesn,doesn't,hadn,hadn't,hasn,hasn't,haven,haven't,isn,isn't,ma,mightn,mightn't,mustn,mustn't,needn,needn't,shan,shan't,shouldn,shouldn't,wasn,wasn't,weren,weren't,won,won't,wouldn,wouldn't"

In [40]:
stopWords=set(stopwords.words("english"))

def remove_stopword(text):
    return " ".join([word for word in str(text).split() if word not in stopWords])

df_text["text_to_stop"]=df_text['text'].apply(lambda text: remove_stopword(text))

#결과 확인
df_text.head()

<ipython-input-40-6f99d52c0b88>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text["text_to_stop"]=df_text['text'].apply(lambda text: remove_stopword(text))


,text,lower_text,text_to_punc,text_to_stop
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 I understand I would like to assist you...,@115712 I understand. I would like assist you....
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,@sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare I have sent several private message...,@sprintcare I sent several private messages on...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 Please send us a Private Message so tha...,@115712 Please send us Private Message assist ...
4,@sprintcare I did.,@sprintcare i did.,sprintcare I did,@sprintcare I did.


In [41]:
# frequent word 확인
from collections import Counter
cnt=Counter()

for text in df_text['text_to_stop'].values:
    for word in text.split():
        cnt[word]+=1

cnt.most_common(10)

[('I', 830261),
 ('us', 395120),
 ('DM', 279259),
 ('Please', 229142),
 ('We', 211563),
 ('get', 192614),
 ('Hi', 147378),
 ('please', 142142),
 ('like', 140317),
 ('@AmazonHelp', 135675)]

In [42]:
# frequent words 제거
frequentWords = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in frequentWords])

df_text["text_wo_stopfreq"] = df_text["text_to_stop"].apply(lambda text: remove_freqwords(text))

# 결과 확인
df_text.head()

<ipython-input-42-48c671085e6c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text["text_wo_stopfreq"] = df_text["text_to_stop"].apply(lambda text: remove_freqwords(text))


,text,lower_text,text_to_punc,text_to_stop,text_wo_stopfreq
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 I understand I would like to assist you...,@115712 I understand. I would like assist you....,@115712 understand. would assist you. would ne...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,@sprintcare propose,@sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare I have sent several private message...,@sprintcare I sent several private messages on...,@sprintcare sent several private messages one ...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 Please send us a Private Message so tha...,@115712 Please send us Private Message assist ...,@115712 send Private Message assist you. Just ...
4,@sprintcare I did.,@sprintcare i did.,sprintcare I did,@sprintcare I did.,@sprintcare did.


In [43]:
from collections import Counter

# Assuming df_text is your DataFrame containing the 'text' column

# Convert text to lowercase
df_text['lower_text'] = df_text['text'].str.lower()

# Punctuation removal function
punc_to_remove = string.punctuation

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punc_to_remove))

# Remove punctuation from 'lower_text'
df_text["text_to_punc"] = df_text['lower_text'].apply(remove_punctuation)

# Download stopwords and remove them from the text
nltk.download('stopwords')
stopWords = set(stopwords.words("english"))

def remove_stopword(text):
    return " ".join([word for word in str(text).split() if word not in stopWords])

df_text["text_wo_stop"] = df_text['text_to_punc'].apply(remove_stopword)

# Count the most frequent words and remove them
cnt = Counter()

for text in df_text['text_wo_stop'].values:
    for word in text.split():
        cnt[word] += 1

frequentWords = set([w for (w, wc) in cnt.most_common(10)])

def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in frequentWords])

df_text["text_wo_stopfreq"] = df_text["text_wo_stop"].apply(remove_freqwords)

# The final preprocessed text will be in the 'text_wo_stopfreq' column
df_text.head()


<ipython-input-43-c1a347b69572>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['lower_text'] = df_text['text'].str.lower()
<ipython-input-43-c1a347b69572>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text["text_to_punc"] = df_text['lower_text'].apply(remove_punctuation)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-43-c1a347b69572>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,text,lower_text,text_to_punc,text_to_stop,text_wo_stopfreq,text_wo_stop
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,@115712 I understand. I would like assist you....,115712 understand would assist would need priv...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,@sprintcare propose,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,@sprintcare I sent several private messages on...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,@115712 Please send us Private Message assist ...,115712 send private message assist click ‘mess...,115712 please send us private message assist c...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,@sprintcare I did.,sprintcare,sprintcare


In [44]:
#lowering, remove punctuation stopword frequent word 모두 적용
print(df_text['text_wo_stopfreq'])

0          115712 understand would assist would need priv...
1                                         sprintcare propose
2          sprintcare sent several private messages one r...
3          115712 send private message assist click ‘mess...
4                                                 sprintcare
                                 ...                        
2811769    823869 hey wed happy look send direct message ...
2811770    115714 wtf i’ve really shitty service day shit...
2811771    143549 sprintcare go httpstcov2tmhetl7q ask ad...
2811772    823870 sounds delicious sarah 😋 httpstco7uqpwy...
2811773    aldiuk warm sloe gin mince pies ice cream best...
Name: text_wo_stopfreq, Length: 2811774, dtype: object


#모델

In [24]:
#OpenAI api 활용
api_key = 'secret key'
openai.api_key = api_key

MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "이 문장을 토큰화 해줄래?  문장 : '아 이 영화 정말 재미 없네' "},
    ],
    temperature=0,
)

In [25]:
print(response['choices'][0]['message']['content'])

['아', '이', '영화', '정말', '재미', '없네']
